# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

modeling_chatglm.py:   0%|          | 0.00/55.7k [00:00<?, ?B/s]

quantization.py:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

tokenizer_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

tokenization_chatglm.py:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

C:\Users\16014\.cache\huggingface\modules\transformers_modules\THUDM\chatglm3-6b\b098244a71fbe69ce149682d9072a7629f7e908c\modeling_chatglm.py:226: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(query_layer, key_layer, value_layer,


In [7]:
print(response)

乾卦是八卦之一，也是八宫图之一，其卦象由三个阳爻夹一个阴爻构成，象征着天、云和雷等自然现象。乾卦的卦辞为“元、亨、利、贞”，意味着“元始、亨通、有利、正定”，它是一种积极向上、充满生机的象征。

乾卦代表了阳、刚、强、动、创造、领导等特质。在五行中，乾卦对应于金，代表着秋、收获、坚固、不动等。在中医养生中，乾卦与脾胃、肾脏等功能相关，与大地、坚固、稳固等特质也有关联。

乾卦在周易六十四卦中具有重要的地位，可以单独出现，也可以与其他卦组合出现。它不仅是一种哲学思想，同时也广泛应用于医学、农业、建筑、军事等领域。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是六十四卦中的第三卦。它的卦象由两个阳爻夹一个阴爻构成，象征着天、云和雷等自然现象。讼卦的卦辞为“元、亨、利、贞”，意味着“元始、亨通、有利、正定”，它是一种积极向上、充满生机的象征。

讼卦代表了阳、刚、强、动、创造、领导等特质。在五行中，讼卦对应于火，代表着夏季、炎热、变动、矛盾等。在中医养生中，讼卦与心脏、肺脏等功能相关，与情感、矛盾、战争等也有关联。

讼卦在周易六十四卦中具有重要地位，可以单独出现，也可以与其他卦组合出现。它不仅是一种哲学思想，同时也广泛应用于法律、诉讼、争端、矛盾等领域。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240406_125824"

#peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"
peft_model_path=f"models/THUDM/chatglm3-6b-epoch3-20240406_130157/checkpoint-60"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [11]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [12]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，由两个离字叠加而成，代表天。它象征着君子应当像天那样刚正不阿，像天那样刚中有柔，像天那样平等无私，像天那样坚定不变。乾卦的核心理念是：天行健，君子拟天。君子应当效法天行健，即不断求进，臀部靠近腿部，走遍天下。因此，乾卦既代表止于至善，也代表不断求进。在事业上，君子应当法天行健，努力进取，不变不化，坚贞正义。在经商中，经商者应如天行健，追求利益，同时遵守道德规范。在求知中，学者应如天行健，不断求进，探索真理。在婚恋中，夫妻双方应当平等对待，相互信任，如同天行健。在运势方面，运势专用，寻求好的风水以及选择吉日等。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240406_125824）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [13]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240406_125824）：
[gMASK]sop 周易中的讼卦是什么样子? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [14]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
读取卦象时，需将取卦者置身于室内，然后抛出硬币或使用卦杯，观察铜钱落地的方向，以决定卦象。根据铜钱落地的正反方向，配合卦象的卦辞，即可解读出卦象及应对策略。师卦是《易经》中的一种卦象，由下卦乾和上卦坎组成。特点是流露出大众心理, 以及军队、民众等众志成城的力量。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240406_125824）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [15]:
from peft import PeftModel, PeftConfig

epochs = 3
#peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"
peft_model_path = f"models/THUDM/chatglm3-6b-epoch3-20240406_130157/checkpoint-50"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [16]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个阳爻夹一个阴爻构成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [17]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
{'name': '地下流水的卦象名为“坎”,卦辞为：“索渊索源，上下其手。” 这一卦象代表着变化和适应，适应不同的环境和条件，适应环境的变化，从而获得成功。', 'content': '\n坎卦是由两个卦象合而成，一个是乾卦，代表天，另一个是坤卦， represent地。合起来象征着天地的相应，形成一种完美的和谐。因此，坎卦象征着智慧和美，代表着最终的成功和美好的结果。\n\n在事业方面，坎卦象征着事业的发展和成长。适应环境的变通和适应能力，能够取得成就，但也有潜在的危机。在商业方面，象征着商场的变化和适应能力，需要及时应对，适应环境的变化。在婚恋方面，象征着爱情的变化和适应能力，需要包容与沟通，维持和谐的关系。\n\n总结起来，坎卦代表着变化和适应，提醒人们要适应环境的变化，适应变通，才能取得成功。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 您说的地水师卦可能指的是地水师卦，是一种卦象，由下卦乾，上卦坤组成。它代表着地行动力，主卦代表着地，客卦代表着水。在这个卦象中，地行动力的主卦指数为下卦乾，即，指数为1，2，3，4。地水师卦的卦象是由乾卦和坤卦相互组合而成，代表着地行动力的加强。

地水师卦的核心哲学是：地动摇荡，地水相互影响，行动摇荡不安，需要谋略解决。这一卦象预示着行动会受到阻碍，需要地主的耐心和智慧来应对。

地水师卦的运势指数为：

- 好运指数：8
- 好运指数：7
- 好运指数：6
- 好运指数：5
- 好运指数：4
- 好运指数：3
- 好运指数：2
- 好运指数：1

地水师卦的建议：地水师卦提示你，地动摇荡，地水相互影响，行动摇荡不安。因此，在这一卦象中，需要耐心和智慧来应对。需要特别注意地动的影响，避免过于冲动行事，应积极应对，避免受到阻碍。同时，也需要注意地面的湿滑，避免摔倒受伤。


In [18]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么样子? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [21]:
from peft import PeftModel, PeftConfig

epochs = 40
#peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"
peft_model_path = f"models/THUDM/chatglm3-6b-epoch3-20240406_130157/checkpoint-40"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [22]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个乾卦叠加而成，代表天。它所代表的是刚健、健行、刚健不屈的意境。在周易中，乾卦代表着君子Model，提示人们要像天一样刚健，像君子一样行事。乾卦的核心哲学是：天行健，君子自强不息。', 'content': '\n乾卦象征天，代表着君子应该像天一样刚健，自强不息。它提醒人们要坚定信念，努力奋斗，永不放弃，从而达到omni-present and all-powerful status. 在事业、经商、求名等方面，乾卦都代表着君子应该自强不息，健行不息，坚贞不屈，不可战胜。在婚恋方面，乾卦提示人们要坚定贞洁，严守道德底线。在决策方面，乾卦提示人们要自强不息，坚定刚健，公正无私，坚强不屈。'}


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [23]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
{'name': '震卦下，坤卦上，震雷动，振作兴盛，激发活力。在这个卦象中，激发的动力和活力被寓意为雷，而卦象本身则代表著水。因此，地水师卦象征着活力和智慧之流，如同流水般涌动，充满生机。', 'content': '\n在这个卦中，君子应当以雷雨之助，积极行事，展开活动，实施计划。雷雨之助，是为了激发活力，振作兴盛，激活生机。而卦象中的卦象，也是为了提醒人们要积极行动，不能等待时机。\n\n地水师卦中， shaking 卦象 显示着整理毛发，而 Interpretation 卦象则表示整合、整理。这表明，在整理、整理工作方面，最好的做法是行动迅速，整合资源，发挥合力。\n\n地水师卦中，复原卦显示着完成某种行动，而寒冰卦则表示等待时机。这表明，在实施计划的过程中，需要有耐心，等待最佳时机，不要匆忙行事。\n\n地水师卦中，姤卦表示错误的时间和错误的机会，而观卦则表示若捉住时机，就能获得成功的经验。这表明，若能在行动中掌握时机，整合资源，就能获得成功。\n\n地水师卦中，若是，就吉，若否，就凶。这表明，若能在行动中掌握时机，积极行事，就能获得吉祥，若吉祥，就吉利，若不吉祥，就招凶。'}


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 初爻：乾为天，初爻复得乾 象。阳爻居下位，象势偏盛，有刚健之志。此爻之时，宜刚健自强，求刚健之志，行刚健之政。

此爻之吉凶取决于下方是否得力，如果下方顺利，则吉利。若下方艰难，则不利。若自身努力自强，则可吉险兼顾。若求得吉利，则需得贤能之佐助，必须努力求得人才。若需行动，则需勇敢果断，充满自信。


In [24]:
base_response, ft_response = compare_chatglm_results("请详细讲讲天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：请详细讲讲天水讼卦

原始输出：
{'name': 'ocal卦是周易中的一卦，由两个震卦叠加而成，代表了一种和谐均衡的美。在这个卦象中，预示着事情的开始是顺利的，但若不过于自信和急功近利，则容易招引是非。和组织团队要注意团结协作，遵守秩序，行动要稳定、稳重。', 'content': '\n讼卦预示着君子们在处理诉讼案件时，应当遵循法律程序，以理性和公平的态度处理问题。虽然开始顺利，但需警惕锋芒所指的方向，避免卷入争端。如果在工作中坚持不懈，则可获得完美的解决。对于求名章士，需要诚实正直，注重名节。对于婚恋，需要明理顺从，避免出现矛盾。\n\n讼卦的教育意义是，做事要公正、公平，要耐心坚持，不能急功近利，要自信和冷静，避免纠纷，保持冷静和理智，才能形成 perfect solution。在事业上，要诚实正直，坚持正道，虽然 initially顺利，但需警惕，避免卷入争端。在求名、婚恋方面，要明理顺从，保持理智，避免矛盾。'}


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 请详细讲讲天水讼卦 讼卦是一个充满变数的卦象，它由上卦乾（天）和下卦坎（水）组成。在这个卦象中，代表天和水的元素相互融合，形成一个充满变数的卦象。因此，在这个卦中，预示着丰富多变的机遇和冒险。虽然上卦乾象征天，代表着稳定和刚毅，但下卦坎代表着险阻和困难。因此，尽管有稳定因素，但风险仍然存在。对于求解者来说，需要在冒险和保守之间做出权衡。总的来说，讼卦是由天和水的混合组成的充满变数的卦象，预示着稳定和风险并存。对于求解者来说，需要在冒险和保守之间做出权衡，并在决策时充分考虑风险和机遇。
